In [1]:
from google import genai
from google.genai import types

In [2]:
# Configure the client
client = genai.Client()

In [3]:
# Define the grounding tool
grounding_tool = types.Tool(
    google_search=types.GoogleSearch()
)

In [4]:
# Configure generation settings
config = types.GenerateContentConfig(
    tools=[grounding_tool]
)

In [5]:
# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Which company had higher revenues in 2024? Uber or Lyft?",
    config=config,
)

In [ ]:
def add_citations(response):
    text = response.text
    supports = response.candidates[0].grounding_metadata.grounding_supports
    chunks = response.candidates[0].grounding_metadata.grounding_chunks

    # Sort supports by end_index in descending order to avoid shifting issues when inserting.
    sorted_supports = sorted(supports, key=lambda s: s.segment.end_index, reverse=True)

    for support in sorted_supports:
        end_index = support.segment.end_index
        if support.grounding_chunk_indices:
            # Create citation string like [1](link1)[2](link2)
            citation_links = []
            for i in support.grounding_chunk_indices:
                if i < len(chunks):
                    uri = chunks[i].web.uri
                    citation_links.append(f"[{i + 1}]({uri})")

            citation_string = "\n ".join(citation_links)
            text = text[:end_index] + citation_string + text[end_index:]

    return text

# Assuming response with grounding metadata
text_with_citations = add_citations(response)
print(text_with_citations)

In 2024, Uber had significantly higher revenues than Lyft. Uber generated $43.9 billion in revenue, an 18% increase year-on-year.[1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGs8v0zMKkMKWGQEyWEGtf1S1gMFgRc3BgjEKRKrEOfFX_XU4z4wUcERAgdjSVKixZXcTsd55jrYIze0CdOhV2c5801f7zTNj1JvEeEV7O44LhrM9wDsiedIzrZRJc3UZ_8JjE2t5J1F5EJWu2cwg==)
 [2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHSjlPBSSNmKHN01iWRZ_hEI4jhRqGwNy_ao0_QygfP30LS2d14Z39VBKIak5bXXPqu69yG_nOUguXLHIYoNa8zcjUMNOfD9fqaADJ0ijz7rzXAFGIGRejMPOVSDSGpaka3v2cIPY8PMvU6nk4H_9KDy8f_BlD2C-Mtr8CsKwVuki6t)
 [3](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEBb3ahEdW3Hp6ceXxPcC766mTJTNytXC4t1HraDSFbS8xYnpHRjRpwsjxuXDmYZt5N0nSlowf-VLQK_R0xZi9IsNMoGDFvMnTalbBHjF4Mghy_QnZR5N6bqf7fAtubFUI7TUWsOtg=) In contrast, Lyft's revenue for 2024 was reported to be between $5.7 billion and $5.8 billion.[4](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH7_5b5jlqQYMwk6